## VQE, new syntax

Very convenient, because the user now has direct access to the inner workings of VQE (no need for high-level function that hides the details.

In [1]:
import numpy as np
from qat.fermion.jordan_wigner import jordan_wigner_representation
from qat.fermion.hamiltonian import FermionicHamiltonian
from qat.lang.AQASM import H, RX, RY, CNOT, QRoutine
from qat.core import Observable, Term
from qat.plugins import ObservableSplitter
from qat.linalg import get_qpu_server
from qat.lang.AQASM import Program
from qat.fermion.hamiltonian import convert_to_observable
import scipy.optimize
from qat.noisy import NoisyQProc

U = 2.0
nqbit = 2
hpq = np.zeros((nqbit, nqbit))
hpq[0,0] = hpq[1,1] = - U/2
hpqrs = np.zeros((nqbit, nqbit, nqbit, nqbit))
hpqrs[0, 1, 0, 1] = hpqrs[1, 0, 1, 0] = - U

hamilt = FermionicHamiltonian(hpq=hpq, hpqrs=hpqrs, mapping=jordan_wigner_representation)

def simple_circuit_with_two_parameters(theta):
    """Take a parameter theta and return the corresponding circuit"""
    Qrout = QRoutine()
    Qrout.apply(H,0)
    Qrout.apply(RY(theta[0]),0)
    Qrout.apply(CNOT, 0,1)
    Qrout.apply(RX(theta[1]),1)
    return Qrout

linalg_qpu = get_qpu_server()
obs = convert_to_observable(hamilt)

def fun(theta, nbshots, qpu):
    prog = Program()
    reg = prog.qalloc(obs.nbqbits)
    prog.apply(simple_circuit_with_two_parameters(theta), reg)
    job = prog.to_circ().to_job(job_type="OBS", observable=obs, nbshots=nbshots)
    res = qpu.submit(job)
    return res.value


We now choose a QPU and run the VQE optimization loop:

In [2]:
%%time
#qpu = ObservableSplitter(method="naive") | linalg_qpu
#qpu = linalg_qpu
#qpu = NoisyQProc(sim_method='deterministic-vectorized')
qpu = ObservableSplitter(method="naive") | NoisyQProc()
theta0 = [0.5, 1.23]
res = scipy.optimize.minimize(lambda theta: fun(theta, 0, qpu), theta0)
print(res.x, res.fun) #exact result = -1

[0.49999997 3.14159264] -0.9999999999999998
CPU times: user 3.79 s, sys: 538 ms, total: 4.33 s
Wall time: 619 ms


## An example using QLM's optimizers (SPSA, Nelder-Mead)

In [3]:
# to do

## Using ``qat.vsolve``

In [4]:
import numpy as np
from qat.fermion.jordan_wigner import jordan_wigner_representation
from qat.fermion.hamiltonian import FermionicHamiltonian
from qat.lang.AQASM import H, RX, RY, CNOT, QRoutine
from qat.core import Observable, Term
from qat.plugins import ObservableSplitter
from qat.linalg import get_qpu_server
from qat.lang.AQASM import Program
from qat.fermion.hamiltonian import convert_to_observable
import scipy.optimize
from qat.noisy import NoisyQProc

U = 2.0
nqbit = 2
hpq = np.zeros((nqbit, nqbit))
hpq[0,0] = hpq[1,1] = - U/2
hpqrs = np.zeros((nqbit, nqbit, nqbit, nqbit))
hpqrs[0, 1, 0, 1] = hpqrs[1, 0, 1, 0] = - U

hamilt = FermionicHamiltonian(hpq=hpq, hpqrs=hpqrs, mapping=jordan_wigner_representation)

def simple_circuit_with_two_parameters(theta):
    """Take a parameter theta and return the corresponding circuit"""
    Qrout = QRoutine()
    Qrout.apply(H,0)
    Qrout.apply(RY(theta[0]),0)
    Qrout.apply(CNOT, 0,1)
    Qrout.apply(RX(theta[1]),1)
    return Qrout

linalg_qpu = get_qpu_server()
obs = convert_to_observable(hamilt)
prog = Program()
reg = prog.qalloc(obs.nbqbits)
prog.apply(simple_circuit_with_two_parameters([prog.new_var(float, 'var_%s'%i) for i in range(2)]), reg)
job = prog.to_circ().to_job(job_type="OBS", observable=obs, nbshots=0)

theta0 = [0.5, 1.23]
from qat.vsolve.optimize import VariationalOptimizer, ScipyMinimize
optimizer = VariationalOptimizer(ScipyMinimize(theta0,
                                               method="COBYLA",
                                               tol=1e-3,
                                               options={"maxiter": 600}))
qpu = optimizer | linalg_qpu


In [5]:
%%time
result = qpu.submit(job)
print(result)

Result(need_flip=False, nbqbits=None, raw_data=None, value=-0.9999996784470463, value_data=None, meta_data={'execution_time': '0.16381621360778809', 'circuit_generation_time': '0.031139373779296875', 'jobs': '29', 'optimization_data': "     fun: -0.9999996784470463\n   maxcv: 0.0\n message: 'Optimization terminated successfully.'\n    nfev: 29\n  status: 1\n success: True\n       x: array([1.51513946, 3.14272677])", 'optimization_trace': '[-0.3328811364377487, -0.33288113643774875, -0.8062437828291925, -0.9980473076030404, -0.8976159752473925, -0.8844161357009999, -0.9582590860686855, -0.9917311924822272, -0.9793683929041168, -0.9996749248621015, -0.9970634837395119, -0.9986786549463575, -0.9982907444349202, -0.9995497326211636, -0.9999940627364594, -0.999833004226545, -0.9999725091493277, -0.9999900559055426, -0.9999165454510057, -0.9999665218504141, -0.9999997632864236, -0.9999979266375773, -0.9999998130443086, -0.999997691639017, -0.9999980292345587, -0.999999995453863, -0.999999680